### Kaggle notebook address: https://www.kaggle.com/code/oliverwang15/4th-solution-ensemble/notebook

In [12]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import gc

In [18]:
CITE_SUBMISSIONS = {
    
    # NN_online 0.89580
    "../input/cite-submissions/submission_nn_online.csv":0.7,

    # CNN 0.8953
    "../input/cite-submissions/submission_cnn/submission":0.7,

    # NN 0.89585
    "../input/cite-submissions/submission_nn/submission":1.2,

    # stacking 0.901
    "../input/cite-submissions/submission_stacking/submission":1.5,

    # kernel_rigde 0.893258
    "../input/cite-submissions/submission_krr/submission":0.5,

    # LGBM 0.8927
    "../input/cite-submissions/submission_lgbm/submission":0.1,
 
    # Catboost 0.8910
    "../input/cite-submissions/submission_catboost/submission":0.1,

}

In [13]:
MULTI_SUBMISSIONS = {
    # stacking 
    "../input/multi-submissions/submission_stacking/submission":1.5,

    # NN 0.6691
    "../input/multi-submissions/submission_nn_1/submission":1,

     # NN 0.67256
    "../input/multi-submissions/submission_nn_2/submission":1,

     # NN 
    "../input/multi-submissions/submission_nn_3/submission":0.5,
    
    # Catboost
    "../input/multi-submissions/submission_catboost.csv": 0.3,

    # LGBM
    "../input/multi-submissions/submission_lgbm/submission":0.1,
    
    # best online
    "../input/multi-submissions/submission_online.csv":0.5

}

In [8]:
best_df = pd.read_csv("../input/cite-submissions/submission_catboost/submission",index_col =0)
best_df.shape

(65744180, 1)

In [6]:
for path, weight in CITE_SUBMISSIONS.items():
    break
best_df = pd.read_csv(path,index_col =0)
best_df.shape

(65744180, 1)

In [3]:
part = [j for i,j in CITE_SUBMISSIONS.items()]
s_part = sum(part)
part = [i/s_part for i in part]
part

[0.14583333333333337,
 0.14583333333333337,
 0.25000000000000006,
 0.31250000000000006,
 0.10416666666666669,
 0.02083333333333334,
 0.02083333333333334]

In [21]:
part = [j for i,j in MULTI_SUBMISSIONS.items()]
s_part = sum(part)
part = [i/s_part for i in part]
part

[0.30612244897959184,
 0.20408163265306126,
 0.20408163265306126,
 0.10204081632653063,
 0.06122448979591837,
 0.020408163265306124,
 0.10204081632653063]

In [5]:
cell_ids = pd.read_csv(r'../input/open-problems-multimodal/evaluation_ids.csv').cell_id

In [14]:
def std(x):
    return (x - np.mean(x)) / np.std(x)

In [15]:
def gen_std_submission(path, cell_ids):
    """
    Standardize submission per cell_id
    """
    df = pd.read_csv(path)
    df['cell_id'] = cell_ids    
    vals = []
    for idx, g in tqdm(df.groupby('cell_id', sort=False), desc=f'Standardizing {path}', miniters=1000):
        vals.append(std(g.target).values)
    del df
    gc.collect()
    vals = np.concatenate(vals)
    return vals

In [16]:
def gen_ensemble(SUBMISSIONS):
    ensemble = None
    for path in tqdm([path for path in SUBMISSIONS.keys()], desc='Process submission'):
        weight = SUBMISSIONS[path]
        if ensemble is None:
            ensemble = gen_std_submission(path, cell_ids) * weight
        else:
            ensemble += gen_std_submission(path, cell_ids) * weight
    gc.collect()
    return ensemble

In [19]:
%%time

PRED_SEGMENTS = [(0, 6812820),(6812820, 65744180) ] #
SUBMISSIONS = [CITE_SUBMISSIONS,MULTI_SUBMISSIONS]
ensemble = []
for sub, (from_idx, to_idx) in tqdm(list(zip(SUBMISSIONS, PRED_SEGMENTS)), desc='Technology'):    
    ensemble.append(gen_ensemble(sub)[from_idx: to_idx])
    
    
ensemble = np.concatenate(ensemble)

Technology:   0%|          | 0/2 [00:00<?, ?it/s]

Process submission:   0%|          | 0/7 [00:00<?, ?it/s]

Standardizing ../input/cite-submissions/submission_nn_online.csv:   0%|          | 0/65443 [00:00<?, ?it/s]

Standardizing ../input/cite-submissions/submission_cnn/submission:   0%|          | 0/65443 [00:00<?, ?it/s]

Standardizing ../input/cite-submissions/submission_nn/submission:   0%|          | 0/65443 [00:00<?, ?it/s]

...

Standardizing ../input/multi-submissions/submission_nn_3/submission:   0%|          | 0/65443 [00:00<?, ?it/s]

Standardizing ../input/multi-submissions/submission_catboost.csv:   0%|          | 0/65443 [00:00<?, ?it/s]

Standardizing ../input/multi-submissions/submission_lgbm/submission:   0%|          | 0/65443 [00:00<?, ?it/s]

Standardizing ../input/multi-submissions/submission_online.csv:   0%|          | 0/65443 [00:00<?, ?it/s]

CPU times: user 17min 59s, sys: 1min 28s, total: 19min 28s
Wall time: 23min 11s


In [20]:
df_submit = pd.read_csv('../input/open-problems-multimodal/sample_submission.csv')
df_submit['target'] = ensemble
print("ensemble -> submission.zip")
df_submit.to_csv('submission.zip', index=False)
df_submit

ensemble -> submission.zip


,row_id,target
0,0,-2.091916
1,1,-1.623264
2,2,-1.523636
3,3,5.333938
4,4,2.404821
...,...,...
65744175,65744175,21.163233
65744176,65744176,-2.931209
65744177,65744177,-2.938488
65744178,65744178,0.965316
